In [7]:
# 这是最简单的 LTSM 模型
import torch
import torch.nn as nn

# 初始化的 lstm 模型
lstm_model = nn.LSTM(10, 20, 2)                    # 参数为(input_size, hidden_size, num_layers)

# 模型的cell接受的输入参数
x = torch.randn(5, 3, 10)                         # 参数为(seq_len, batch, input_size)
# 模型的cell接受的输入hidden_state；也是前一个cell的输出hidden_state
h0 = torch.randn(2, 3, 20)                         # 参数为(num_layers * num_directions, batch, hidden_size)
# 模型的cell初始值
c0 = torch.randn(2, 3, 20)                         # 参数为(num_layers * num_directions, batch, hidden_size)

y, (h1, c1) = lstm_model(x, (h0, c0))

print(y.size())
print(h1.size())
print(c1.size())


torch.Size([5, 3, 20])
torch.Size([2, 3, 20])
torch.Size([2, 3, 20])


In [1]:
# LSTM 做股票预测
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# GPU -- 准备来做一个GPU版本的RNN
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 设置 GPU 优先
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# 加载数据
dataset = pd.read_csv("000600.csv", index_col=0)
dataset = dataset.drop(['date','prediction'], axis=1)
# print(dataset.columns)
# print(dataset.tail())
dataset['updown'] = dataset['updown']
print(dataset.shape)
# print(dataset.tail())

# 方法2，将数据按照batch_size的窗口进行滑动，每个窗口数据做一组

# # 数据转成sequence的格式，这里定义每个seq的长度
seq_len = 21
batch_size = 2                                                    # 注意：batch_size是要能够整除seq_count的

# 把数据切换成 batch_size 的一个个batch
rolling_data = pd.DataFrame()
for i in dataset.rolling(seq_len):
    if i.shape[0] == seq_len:
        rolling_data = rolling_data.append(i)

rolling_data = rolling_data.values.reshape(-1, seq_len, 118)                 # 数据一共是 seq_count x seq_len x in_dim

print("rolling_data: {}".format(rolling_data.shape))
print("seq count: {}".format(rolling_data.shape[0]))                 # 所以一共有 seq_count 列数据，每一行的数据是118维 （包括y）
print("seq length: {}".format(seq_len))


total_batch_count = int(rolling_data.shape[0]/batch_size)                          # 把数据规划成 batch_count 个 batch

print("total batch count: {}".format(total_batch_count))
print("batch size: {}".format(batch_size))

rolling_data = rolling_data.reshape(total_batch_count, batch_size, seq_len, 118)  # 把数据转成 total_batch_count x batch_size x seq_len x in_dim 格式
rolling_data = torch.tensor(rolling_data)
print("rolling_data: {}".format(rolling_data.shape))


train_batch_count = total_batch_count - 1
test_batch_count = total_batch_count - train_batch_count

train = rolling_data[:train_batch_count, :, :, :]
test  = rolling_data[train_batch_count:, :, :, :]

train_x, train_y = train[:,:,:,1:], train[:,:,:,0:1]
test_x,  test_y  = test[:,:,:, 1:],  test[:,:,:,0:1]

train_x = train_x.to(device)
train_y = train_y.to(device)
test_x = test_x.to(device)
test_y = test_y.to(device)

print("train_x: {}".format(train_x.shape))
print("train_y: {}".format(train_y.shape))
print("test_x:  {}".format(test_x.shape))
print("test_y:  {}".format(test_y.shape))
print("train_batch_count: {}".format(train_batch_count))
print("test_batch_count:  {}".format(test_batch_count))

(150, 118)
rolling_data: (130, 21, 118)
seq count: 130
seq length: 21
total batch count: 65
batch size: 2
rolling_data: torch.Size([65, 2, 21, 118])
train_x: torch.Size([64, 2, 21, 117])
train_y: torch.Size([64, 2, 21, 1])
test_x:  torch.Size([1, 2, 21, 117])
test_y:  torch.Size([1, 2, 21, 1])
train_batch_count: 64
test_batch_count:  1


In [7]:
# 开始定义 LSTM 模型
TIME_STEP = seq_len                                        # 一般这个单独设定，这里为了简单，还是直接就等于seq_len的方便。其实也就是等于最长的那个sequence length
INPUT_SIZE = 117
HIDDEN_SIZE = 768

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        
        self.rnn = nn.RNN(
                input_size = INPUT_SIZE,                    # 输入 x 的feature 维度
                hidden_size = HIDDEN_SIZE,                  # 隐状态 hidden_state 中的feature维度
                num_layers = 5,                             # RNN 的层数
                nonlinearity='relu',                        # 指定激活函数 [‘tanh’ | ’relu’]. 默认: ‘tanh’
                bias = True,                                # 如果是 False , 那么 RNN 层就不会使用偏置权重 b_ih 和 b_hh, 默认: True
                batch_first = True,                         # 如果 True, 输入Tensor的shape应该是(batch_size, seq_len, features),并且输出也是一样.
                dropout = 0.1,                                # 如果值非零, 那么除了最后一层外, 其它层的输出都会套上一个 dropout 层
                bidirectional = False                       # 如果 True , 将会变成一个双向 RNN, 默认为 False
        )
        
        self.out = nn.Linear(HIDDEN_SIZE, 1)
        
    def forward(self, x, h_state):
        r_out, h_state = self.rnn(x, h_state)
        outs = []
        for time_step in range(TIME_STEP):
            outs.append(self.out(r_out[:, time_step, :]))

        return torch.stack(outs, dim=1), h_state